<a href="https://colab.research.google.com/github/prakhar-kt/ARIMA-SARIMA/blob/main/Model_%2B_MLE_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



This question has been created to test your statistical analysis and programming knowledge in Python. 

You are given a `csv` file, which include various data entries for each football match in **English Premier League** during the 2020-2021 season. To name a few of these entries: date, referee name, number of goals, red cards, etc. The `csv` dataset you are provided contains one row per football match. The column names are abbreviations and given as: 

```
Div = League Division
Date = Match Date (dd/mm/yy)
Time = Time of match kick off
HomeTeam = Home Team
AwayTeam = Away Team
FTHG = Full Time Home Team Goals
FTAG = Full Time Away Team Goals
FTR = Full Time Result (H=Home Win, D=Draw, A=Away Win)
HTHG = Half Time Home Team Goals
HTAG = Half Time Away Team Goals
HTR = Half Time Result (H=Home Win, D=Draw, A=Away Win)
Referee = Match Referee
HS = Home Team Shots
AS = Away Team Shots
HST = Home Team Shots on Target
AST = Away Team Shots on Target
HF = Home Team Fouls Committed
AF = Away Team Fouls Committed
HC = Home Team Corners
AC = Away Team Corners
HY = Home Team Yellow Cards
AY = Away Team Yellow Cards
HR = Home Team Red Cards
AR = Away Team Red Cards
```


In this exercise, you are asked to perform a number of operations to:

 - perform statistical analysis of the data, and

 - gain insights from the data.

In [66]:
# suggested imports
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy
from urllib import request
import scipy.stats as stats
from statsmodels import graphics
import arviz as az
import pymc3 as pm
from pymc3 import glm
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, RocCurveDisplay, auc, roc_curve
import seaborn as sns
sns.set_style(style="darkgrid", rc={"axes.facecolor": ".9", "grid.color": ".8"})
sns.set_palette(palette="deep")
sns_c = sns.color_palette(palette="deep")

In [67]:
module_url = f"https://raw.githubusercontent.com/oktaykarakus/cmt309-portfolio/main/EPL_season-2021.csv"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

df = pd.read_csv('EPL_season-2021.csv')

Fetching https://raw.githubusercontent.com/oktaykarakus/cmt309-portfolio/main/EPL_season-2021.csv


In [68]:
df.head(5)

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,E0,12/09/2020,12:30,Fulham,Arsenal,0,3,A,0,1,...,2,6,12,12,2,3,2,2,0,0
1,E0,12/09/2020,15:00,Crystal Palace,Southampton,1,0,H,1,0,...,3,5,14,11,7,3,2,1,0,0
2,E0,12/09/2020,17:30,Liverpool,Leeds,4,3,H,3,2,...,6,3,9,6,9,0,1,0,0,0
3,E0,12/09/2020,20:00,West Ham,Newcastle,0,2,A,0,0,...,3,2,13,7,8,7,2,2,0,0
4,E0,13/09/2020,14:00,West Brom,Leicester,0,3,A,0,0,...,1,7,12,9,2,5,1,1,0,0


## P2.1 - Data Pre-processing and Exploratory Analysis 

In this question, your task is to use `pandas` and other required modules to preprocess the data frame, `df`. Preprocessing will include: add/remove/recode columns in `df`. In addition, to further explore the dataset, you need to produce a number of exploratory plots. 

#### P2.1.1 - Add Booking Points Columns (1 marks) 

Sometimes, in order to better analyse any given data set, one can create a new type of feature by combining two or more existing entries of the data frame. In this question, you are asked to create a function `add_booking_pts(df)` which creates two new columns of: **Home booking points (HBP)**, and **Away booking points (ABP)** by using four existing columns of HY, AY, HR, and AR.

The details of the function `add_booking_pts(df)` are given below:

 - Takes the data frame `df` as input.
 
 - For each match, number of yellow cards is weighted with 10 points, whilst the number of red cards is with 25 points. 
 
 - Basically, the function calculates HBP and ABP columns as
    - $HBP = 10\cdot HY + 25\cdot HR$
    - $ABP = 10\cdot AY + 25\cdot AR$
 
 - These newly created arrays are added to `df`, whilst removing the columns for HY, AY, HR, and AR.
 
 - Finally, the updated `df` is returned.

In [70]:
def add_booking_pts(df):
    df['HBP'] = 10*df['HY']+25*df['HR']
    df['ABP'] = 10*df['AY']+25*df['AR']
    df = df.drop(['HY','AY','HR','AR'], axis=1)
    return df
df = add_booking_pts(df)
df.head()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,HS,AS,HST,AST,HF,AF,HC,AC,HBP,ABP
0,E0,12/09/2020,12:30,Fulham,Arsenal,0,3,A,0,1,...,5,13,2,6,12,12,2,3,20,20
1,E0,12/09/2020,15:00,Crystal Palace,Southampton,1,0,H,1,0,...,5,9,3,5,14,11,7,3,20,10
2,E0,12/09/2020,17:30,Liverpool,Leeds,4,3,H,3,2,...,22,6,6,3,9,6,9,0,10,0
3,E0,12/09/2020,20:00,West Ham,Newcastle,0,2,A,0,0,...,15,15,3,2,13,7,8,7,20,20
4,E0,13/09/2020,14:00,West Brom,Leicester,0,3,A,0,0,...,7,13,1,7,12,9,2,5,10,10


#### P2.1.2 - Convert Table Colums into Digits (2 marks) 

When reading in the dataframe, one can see that it contains some textual data which will not be relevant for the numerical analyses in Question 1. Therefore, implement a function `convert_results(df)` 

1. (1 mark) to convert **half-time results (HTR)** and **full-time results (FTR)** into numerical data. The details of the function are given below: 
 
- HTR and FTR columns include string values of `'H'`, `'D'` and `'A'`. These string corresponds to the cases below:
    - `'H'`: Home team win

    - `'D'`: Draw

    - `'A'`: Away team win

- The function `convert_results(df)` will replace `'H'`, `'D'` and `'A'` values with `int` type values of of 1, 0, -1, respectively.

2. (1 mark) to convert **Time** column into `float` type values in interval of $[0, 24)$. Since an hour has 60 minutes, a 15-minute interval corresponds to quarter of an hour (i.e 0.25 hours). Considering this, some examples can be given:
 
- `'12.30'` will be `12.5`, or 
 
- `'18.15'` will be `18.25` or 
 
- `'17.00'` will be `17.0`
 
The function `convert_results(df)` should return the updated data frame `df`.

In [71]:
def convert_results(df):
    df['FTR']= df['FTR'].replace(['H','D','A'],[1,0,-1])
    df['HTR'] = df['HTR'].replace(['H','D','A'],[1,0,-1])
    df['Time'] = pd.to_datetime(df['Time'])
    df['Time'] = df['Time'].dt.hour + df['Time'].dt.minute/60
    return df
df = convert_results(df)
df.head()


,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,HS,AS,HST,AST,HF,AF,HC,AC,HBP,ABP
0,E0,12/09/2020,12.5,Fulham,Arsenal,0,3,-1,0,1,...,5,13,2,6,12,12,2,3,20,20
1,E0,12/09/2020,15.0,Crystal Palace,Southampton,1,0,1,1,0,...,5,9,3,5,14,11,7,3,20,10
2,E0,12/09/2020,17.5,Liverpool,Leeds,4,3,1,3,2,...,22,6,6,3,9,6,9,0,10,0
3,E0,12/09/2020,20.0,West Ham,Newcastle,0,2,-1,0,0,...,15,15,3,2,13,7,8,7,20,20
4,E0,13/09/2020,14.0,West Brom,Leicester,0,3,-1,0,0,...,7,13,1,7,12,9,2,5,10,10


In [72]:
# Check the overall info of the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 22 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Div       380 non-null    object 
 1   Date      380 non-null    object 
 2   Time      380 non-null    float64
 3   HomeTeam  380 non-null    object 
 4   AwayTeam  380 non-null    object 
 5   FTHG      380 non-null    int64  
 6   FTAG      380 non-null    int64  
 7   FTR       380 non-null    int64  
 8   HTHG      380 non-null    int64  
 9   HTAG      380 non-null    int64  
 10  HTR       380 non-null    int64  
 11  Referee   380 non-null    object 
 12  HS        380 non-null    int64  
 13  AS        380 non-null    int64  
 14  HST       380 non-null    int64  
 15  AST       380 non-null    int64  
 16  HF        380 non-null    int64  
 17  AF        380 non-null    int64  
 18  HC        380 non-null    int64  
 19  AC        380 non-null    int64  
 20  HBP       380 non-null    int64 

In [73]:
# check the shape
df.shape

(380, 22)

#### Exploratory data analysis

In [74]:
df.Div.value_counts()

E0    380
Name: Div, dtype: int64

This implies that data is for only one type of division

## P2.2 - Statistical Analysis 

#### P2.2.1 - Model selection for Regression Analysis 

In this question, we construct a regression analyses to investigate how well FTHG (or FTAG) can be predicted from the other variables in the dataframe. The objective of this question is to derive a sparse model (linear and polynomial) with fewer variables. 

#### P2.2.1.1 - Variable Selection for Linear Regression 

In **variable selection** ('variable' means the same as 'predictor'), variables get iteratively added or removed from the regression model. Once finished, the model typically contains only a subset of the original variables. It makes it easier to interpret the model, and in some cases it makes it generalise better to new data. 

To perform variable selection, create a function `select_variable(df, main_pred, main_target, alpha)`, where 
 
 - `main_pred` is a dictionary of variables. For this analysis, firstly, either all Home or Away teams will be marked and the predictors given below will be used

  - Home: [Time, FTR, HTHG, HTR, HS, HST, HF, HC, HBP]
  
  - Away: [Time, FTR, HTAG, HTR, AS, AST, AF, AC, ABP]. 
 
 - `main_target` is the variable for the regression, Home: FTHG (or Away: FTAG)
 
 - `alpha` is the significance level for selecting significant predictors

The function should return

 - `main_pred` is the dictionary which stores the selected subset of initial `main_pred` both for home and away teams, in a format of `main_pred = {'Home': [... selected predictors here ...], 'Away': [... selected predictors here ...]}`.

To calculate regression fits and $p$-values you will use `statsmodels`. The general procedure follows two stages:

 - Stage 1 (adding predictors): you build a model by adding variables one after the other. You keep adding variables that increase the **adjusted $R^2$** value (provided by `statsmodels` package). 
  
  - Start with an empty set of variables
  
  - Fit multiple one-variable regression models. In each iteration, use one of the variables provided in predictors. The variable that leads to the largest increase in adjusted $R^2$ is added to the model.
  
  - Now proceed by adding a second variable into the model. Starting from the remaining variables, again choose the variable that leads to the largest increase in adjusted $R^2$.
  
  - Continue in the same way for the third, fourth, … variable.
  
  - You are finished when there is no variable left that increases adjusted $R^2$.
 
 - Stage 2 (removing non-significant predictors): if any of the utilised predictors are not significant, you need to remove them. Keep removing variables until all variables in the model are significant.

  - Start by fitting a model using the variables that have been added to the model in Stage 1.
  
  - If there is a variable that is not significant, remove the variable with the largest $p$-value and fit the model again with the reduced set of variables.
  
  - Keep removing variables and re-fitting the model until all remaining variables are significant.
  
  - The remaining significant variables are the output of your function.

In [75]:
%xmode Plain
%pdb off

Exception reporting mode: Plain
Automatic pdb calling has been turned OFF


In [76]:
import pdb

In [77]:
def select_variable(df, main_pred, main_target, alpha):
    
    
    # For Home team
    predictors = main_pred['Home']
    y = df[main_target['Home']]

    # Get the list of predictors using adjusted R-squared value

    # Stage 1 : Adding Predictors


    
    var_list = []

    r_squared = 0.0


    marker_1 = True

    while marker_1 == True:

        # print(f"marker_1: {marker_1}")
        # pdb.set_trace()
        # set a varible for comparison of r_squared before and after the 
        # loop is run
        r_squared_before = r_squared
        
        selected_var = None
        
        # loop over the predictors one at a time

        for pred in predictors:
            
            # print(f"\npred: {pred}")
            # initialize a selected variable
            # selected_var = None
        
            # add the pred var to variable list
            var_list.append(pred)
            # print(f"var_list: {var_list}")
            
            # define independent variable X  
            X = df[var_list]

            # add a constant term for linear regession
            # using statsmodel library
            X = sm.add_constant(X)
            
            # fit a Ordinary Least Square Model
            model = sm.OLS(y, X).fit()

            # get the rsquared value
            adj_rsquare = model.rsquared_adj
            # print(f"r_squared_before: {r_squared_before}")
            # print(f"adj_rsquare: {adj_rsquare}")


            
            # if the adj_rquared is greater than previous value of r_squared
            # select the var to be added to the variable list
            # and update the r_squared value
            if adj_rsquare > r_squared:
                selected_var = pred
                r_squared = adj_rsquare
            # print(f"r_square: {r_squared}")
            
           
            
                
            # remobe the single pred from the var_list as it was only for 
            # fitting and checking the model
            var_list.remove(pred)
            

        # append the selected variable 
        # having the highest r_score the variable list, and
        # remove the slected variable from the predictors list
        # as we have already added it to the variable list
        # if its value is not none
        # print(f"\nvar_list: {var_list}")
        # print(f"predictors : {predictors}")
        # print(f"selected_var: {selected_var}")


        if isinstance(selected_var,str):
            predictors.remove(selected_var)
            var_list.append(selected_var)
        # print(f"var_list: {var_list}")
        # print(f"predictors : {predictors}")
        # Get the r_squared value after running the loop
        r_squared_after = r_squared
        # print(f"r_squared_after: {r_squared}")


        # Compare the r_sqaured before and after
        # if both are same , we can break the loop as the score is
        # not improving anymore
        if r_squared_before == r_squared_after:
            marker_1 = False
        

        

        # if the list of the predictors is empty, 
        # stop the loop
        if len(predictors) == 0:
            marker_1 = False

    # define a placeholder of boolean values
    marker_2 = True

    # loop until no more insignificant values exist

    while marker_2 == True:

        # Initial placeholder for a  variable
        var = None

        # define input to the model
        X = df[var_list]
        # print(f"var_list: {var_list}")

        # add the constant term 
        X = sm.add_constant(X)

        # fit the model

        model = sm.OLS(y, X).fit()

        # find the p values for the fitted model
        p_values = model.pvalues
        # print(f"p_valuues : {p_values}")

        # initialize the max_p_value equal 
        # to the significance level alpha
        max_p_value = alpha

        # loop over the p_values and 
        # remove the one with the highest value

        for value in p_values[1:]:
            # print(f"\nvalue:{value}")

            if value > max_p_value:
                var = p_values[p_values == value].index[0]
                # print(f"var:{var}")
                max_p_value = value

        # if the value of var is not None, which
        # means that a insignificant variabel was found
        # and can be removed from the list of variables
        if isinstance(var, str):
            var_list.remove(var)

        # else if no insignificant variables where 
        # found, we can exit the loop

        else:
            marker_2 = False


    main_pred['Home'] = var_list
    
   
    # For Away team
    predictors = main_pred['Away']
    y = df[main_target['Away']]

    # Get the list of predictors using adjusted R-squared value

    # Stage 1 : Adding Predictors


    
    var_list = []

    r_squared = 0.0


    marker_1 = True

    while marker_1 == True:

        # print(f"marker_1: {marker_1}")
        # pdb.set_trace()
        # set a varible for comparison of r_squared before and after the 
        # loop is run
        r_squared_before = r_squared
        
        selected_var = None
        
        # loop over the predictors one at a time

        for pred in predictors:
            
            # print(f"\npred: {pred}")
            # initialize a selected variable
            # selected_var = None
        
            # add the pred var to variable list
            var_list.append(pred)
            # print(f"var_list: {var_list}")
            
            # define independent variable X  
            X = df[var_list]

            # add a constant term for linear regession
            # using statsmodel library
            X = sm.add_constant(X)
            
            # fit a Ordinary Least Square Model
            model = sm.OLS(y, X).fit()

            # get the rsquared value
            adj_rsquare = model.rsquared_adj
            # print(f"r_squared_before: {r_squared_before}")
            # print(f"adj_rsquare: {adj_rsquare}")


            
            # if the adj_rquared is greater than previous value of r_squared
            # select the var to be added to the variable list
            # and update the r_squared value
            if adj_rsquare > r_squared:
                selected_var = pred
                r_squared = adj_rsquare
            # print(f"r_square: {r_squared}")
            
           
            
                
            # remobe the single pred from the var_list as it was only for 
            # fitting and checking the model
            var_list.remove(pred)
            

        # append the selected variable 
        # having the highest r_score the variable list, and
        # remove the slected variable from the predictors list
        # as we have already added it to the variable list
        # if its value is not none
        # print(f"\nvar_list: {var_list}")
        # print(f"predictors : {predictors}")
        # print(f"selected_var: {selected_var}")


        if isinstance(selected_var,str):
            predictors.remove(selected_var)
            var_list.append(selected_var)
        # print(f"var_list: {var_list}")
        # print(f"predictors : {predictors}")
        # Get the r_squared value after running the loop
        r_squared_after = r_squared
        # print(f"r_squared_after: {r_squared}")


        # Compare the r_sqaured before and after
        # if both are same , we can break the loop as the score is
        # not improving anymore
        if r_squared_before == r_squared_after:
            marker_1 = False
        

        

        # if the list of the predictors is empty, 
        # stop the loop
        if len(predictors) == 0:
            marker_1 = False

    # define a placeholder of boolean values
    marker_2 = True

    # loop until no more insignificant values exist

    while marker_2 == True:

        # Initial placeholder for a  variable
        var = None

        # define input to the model
        X = df[var_list]
        # print(f"var_list: {var_list}")

        # add the constant term 
        X = sm.add_constant(X)

        # fit the model

        model = sm.OLS(y, X).fit()

        # find the p values for the fitted model
        p_values = model.pvalues
        # print(f"p_valuues : {p_values}")

        # initialize the max_p_value equal 
        # to the significance level alpha
        max_p_value = alpha

        # loop over the p_values and 
        # remove the one with the highest value

        for value in p_values[1:]:
            # print(f"\nvalue:{value}")

            if value > max_p_value:
                var = p_values[p_values == value].index[0]
                # print(f"var:{var}")
                max_p_value = value

        # if the value of var is not None, which
        # means that a insignificant variabel was found
        # and can be removed from the list of variables
        if isinstance(var, str):
            var_list.remove(var)

        # else if no insignificant variables where 
        # found, we can exit the loop

        else:
            marker_2 = False


    main_pred['Away'] = var_list
    
    return main_pred



#### P2.2.1.2 - Model Selection for Polynomial Regression 

Often the dataset provided is not linearly separable and a simple linear regression model may not be able to derive relationships between both the independent and dependent variables. In such cases, a possible solution would be to implement polynomial regression instead (https://en.wikipedia.org/wiki/Polynomial_regression). Polynomial regression is a form of regression analysis in which the relationship between the independent variable $x$ and the dependent variable $y$ is modelled as an $n^{th}$ degree polynomial in $x$.

**Example:** Given $y$ the dependent variable, $x_1, x_2$ the independent variables, $b_0$ the bias and $b_1,b_2,...,b_n$ the weights a polynomial regression of degree 2 would have the form:

$$y = b_0 + b_1x_1 + b_2x_1^2 + b_3x_2 + b_4x_2^2$$

Implement a function `polynomial_model(df, main_pred, main_target, degrees)` which uses the selected subset of variables as an argument from the function `select_variable()`, and calculates all possible combinations of the variable set and polynomial degrees. The function `polynomial_model()` finds the degree that yields the best polynomial model (according to the adjusted R-squared metric) to predict the value of a FTHG or FTAG as in the linear regression part above.

Arguments and outputs of the function are given as

 - a dataframe `df`, 

 - a dictionary `main_pred` indicating the predictors for home and away, 
 
 - a dictionary `main_target` indicating target variable for home and away, 
 
 - a list of integers indicating the degrees to test degrees, 
 
The function should return 

 - the best fitted regression model, and best polynomial degree for home and away in a dictionary `main_predp` of format `main_predp = {'Home': (best_fit, best_degree), 'Away': (best_fit, best_degree)}`..

In [78]:
def polynomial_model(df, main_pred, main_target, degrees):
    # your code here

    # define the output dictionary
    main_predp = {'Home':(),'Away':()}
    
    # For home target variable
    
    y = df[main_target['Home']]
    var_list = main_pred['Home']
    X = df[var_list]

    
    # iterate over the list of degrees
    # to create the columns by expanding the columns
    # to the degree specified

    r_squared = 0

    for degree in degrees:
        if degree == 0:
            continue
        
        # Add columns having polynomial features

        for deg in range(2,degree+1):
            for col in X.columns:
                X[f"{col}**{deg}"] = X[col].apply(lambda x : x**deg)
                # X[f"{col}**{deg}"] = x**deg

        # Scale the columns using min-max scaling

        for col in X.columns:
                max = X[col].max()
                min = X[col].min()
                if max > min:
                    X[col] = X[col].apply(lambda x : (x-min)/(max-min))
        # print(f"degree:{degree}")
        # print(X.head())
    
        # add the constant term 

        X = sm.add_constant(X)

        # fit the model

        model = sm.OLS(y, X).fit()

        adj_r_squared = model.rsquared_adj

        if adj_r_squared > r_squared:
            best_fit = model
            best_degree = degree
            r_squared = adj_r_squared

    
    main_predp['Home'] = (best_fit, best_degree)


    # for the away variable
       
    y = df[main_target['Away']]
    var_list = main_pred['Away']
    X = df[var_list]

    
    # iterate over the list of degrees
    # to create the columns by expanding the columns
    # to the degree specified

    r_squared = 0

    for degree in degrees:
        if degree == 0:
            continue
        
        for deg in range(2,degree+1):
            for col in X.columns:
                X[f"{col}**{deg}"] = X[col].apply(lambda x : x**deg)

        # Scale the columns using min-max scaling

        for col in X.columns:
                max = X[col].max()
                min = X[col].min()
                if max > min:
                    X[col] = X[col].apply(lambda x : (x-min)/(max-min))

    

        # add the constant term 

        X = sm.add_constant(X)

        # fit the model

        model = sm.OLS(y, X).fit()

        adj_r_squared = model.rsquared_adj

        if adj_r_squared > r_squared:
            best_fit = model
            best_degree = degree
            r_squared = adj_r_squared

    

    main_predp['Away'] = (best_fit, best_degree)



    return main_predp

#### P2.2.2 - Predicting Match Result 


Create a function `predict_result()` which predicts the result of **Man City - Everton** football match which was played on 23/05/2021. In order to do this, firstly crop last last 10 rows of the data frame `df` to use only the first 37 weeks (370 matches) of the season to fit your regressors.

The function `predict_result()` will use `select_variable()` and `polynomial_model()` function outputs as the best linear and polynomial regression models. Then by using these two models, it predicts the number of goals scored by Home and Away teams separately, which will lead to the result of the match. Finally, print the information below:

```
Linear regression prediction        : Man City x - y Everton
Polynomial regression prediction    : Man City a - b Everton
Correct result                      : Man City 5 - 0 Everton
```

In [108]:
def predict_result(df, main_pred, main_predp, team1 = 'Man City', team2 = 'Everton'):
    # your code here
    
    df_train = df[:-10]
    df_test = df[-10:]
    # For home team
    var_list = main_pred['Home']
    X = df[var_list]
    
    y = df['FTHG']
    X_test = df_test[(df['Date'] == '23/05/2021') & (df['HomeTeam'] == 'Man City') & (df['AwayTeam'] == 'Everton')][var_list]
    
    print(f"X_test: {X_test}")
    print(f"Type(X_test):{type(X_test)}, X_test.shape:{X_test.shape}")
    print(f"Type(X): {type(X)}, X.shape:{X.shape}")

    # Linear Model:
    # X = sm.add_constant(X)
    linear_model = sm.OLS(y,X).fit()
    # X_test_t = X_test.T.iloc[:,0]
    linear_result_home = linear_model.predict(X_test)

    # Polynomial Model
    poly_model = main_predp['Home'][0]
    degree = main_predp['Home'][1]

    # transform X_test to the same form as X_train
    
    for deg in range(2,degree+1):
        for col in X_test.columns:
            X_test[f"{col}**{deg}"] = X_test[col].apply(lambda x : x**deg)

    # Scale the columns using min-max scaling

    # for col in X_.columns:
    #         max = X[col].max()
    #         min = X[col].min()
    #         if max > min:
    #             X[col] = X[col].apply(lambda x : (x-min)/(max-min))

    poly_result_home = poly_model.predict(X_test)


    # For Away Team
    var_list = main_pred['Away']
    X = df[var_list]
    y = df['FTAG']
    X_test = df_test[(df['Date'] == '23/05/2021') & (df['HomeTeam'] == 'Man City') & (df['AwayTeam'] == 'Everton')][var_list]
    
    # Linear Model:
    # X = sm.add_constant(X)
    linear_model = sm.OLS(y,X).fit()
    # X_test_t = X_test.T.iloc[:,0]
    linear_result_away = linear_model.predict(X_test)
    


    # Polynomial Model
    poly_model = main_predp['Home'][0]
    degree = main_predp['Home'][1]

    for deg in range(2,degree+1):
        for col in X_test.columns:
            X_test[f"{col}**{deg}"] = X_test[col].apply(lambda x : x**deg)

    
    poly_result_away = poly_model.predict(X_test)
    
    results = {'Linear':{'Home':linear_result_home,
                         'Away': linear_result_away},
               'Polynomial' : {'Home': poly_result_home,
                               'Away': poly_result_away}
               }

    return results

    


In [92]:
import warnings
warnings.filterwarnings('ignore')

In [93]:
df_train = df[:-10]
df_train.shape

(370, 22)

In [94]:
df_train.head(2)

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,HS,AS,HST,AST,HF,AF,HC,AC,HBP,ABP
0,E0,12/09/2020,12.5,Fulham,Arsenal,0,3,-1,0,1,...,5,13,2,6,12,12,2,3,20,20
1,E0,12/09/2020,15.0,Crystal Palace,Southampton,1,0,1,1,0,...,5,9,3,5,14,11,7,3,20,10


In [95]:
alpha = 0.05
main_target = {'Home':'FTHG','Away':'FTAG'}
main_pred = {'Home': ['Time', 'FTR', 'HTHG', 'HTR', 'HS', 'HST', 'HF', 'HC', 'HBP'],
             'Away': ['Time', 'FTR', 'HTAG', 'HTR', 'AS', 'AST', 'AF', 'AC', 'ABP']}
main_pred = select_variable(df_train, main_pred, main_target, alpha)
main_pred

{'Away': ['HTAG', 'FTR', 'AST', 'HTR'],
 'Home': ['HTHG', 'HST', 'FTR', 'HTR', 'HC']}

In [96]:
main_predp = polynomial_model(df_train, main_pred, main_target, [2,3,4,5])

In [97]:
main_predp

{'Away': (<statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7f2eb8580dd0>,
  3),
 'Home': (<statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7f2eb9096350>,
  4)}

In [109]:
results = predict_result(df, main_pred, main_predp)

X_test:      HTHG  HST  FTR  HTR  HC
376     2   11    1    1   7
Type(X_test):<class 'pandas.core.frame.DataFrame'>, X_test.shape:(1, 5)
Type(X): <class 'pandas.core.frame.DataFrame'>, X.shape:(380, 5)


ValueError: ignored

#### P2.2.3 - Maximum likelihood estimation (MLE) and prediction 

In this question, you are expected to solve a regression problem, but this time using **maximum likelihood estimation (MLE)** theory. You need to construct a regression analysis to investigate how well the **full time results (FTR)** can be predicted from the other variables of FTHG, HS, HC, AS, AC. 

#### P2.2.3.1 - ML Estimate of regression parameters 

Create a function `ML_estimate(df_est, pred, target)` which calculates the ordinary least squares (OLS) and two MLE fits (Poisson and Probit) for the given arguments.

You need to use `statsmodels` module and its corresponding methods of `.OLS()`, `.Poisson()` and `.Probit()`.

- `df_est` is a subset of the data frame `df` which includes **randomly selected** 280 rows of `df`. The remaining 100 rows will be used in prediction application (see below).

- `pred` is a list of variables. For this analysis, OLS and other models utilise the predictors of FTHG, HS, HC, AS, AC. (Note: Depending on your implementation, you might need to add a constant to the predictors. Please see the lecture notes)

- `target` is the target variable for the regression, FTR. You need to adjust values of this column for the purpose of this question. 
 
  - FTR = 1.0 if Home team wins.
  
  - FTR = 0.0 if Away team wins or a Draw.

The function should return variables 

 - `MLE_model_fits` a `tuple` object which stores all three model fits `statsmodels` objects for OLS, Probit and Poisson.

In [ ]:
def ML_estimate(df_est, pred, target):
    # your code here
    return MLE_model_fits

#### P2.2.3.2 - Predicting Home Win via MLE 

Create a function `ML_predict(df_pred, MLE_model_fits)` which calculates FTR predictions for all three models of OLS, Probit and Poisson.

You need to use `statsmodels`' method for prediction: `.predict()`.

- `df_pred` is a subset of the data frame `df` which includes only a subset of 100 rows of `df`.

- `MLE_model_fits` is a `tuple` object obtained from the `ML_estimate()` function above. Unpack this argument to obtain `statsmodels` objects for all three models. 

This function should return: 

 - `df_pred`

 - `MLE_model_predictions` is a `tuple` which stores the predicted outputs for each three models.  

In [ ]:
def ML_predict(df_pred, MLE_model_fits):
    # your code here
    return MLE_model_predictions

#### P2.2.4 - Evaluating Prediction Performance 

You will now need to visualise the prediction performance of the models, and evaluate them in terms of prediction accuracy, and mean square error (MSE) metrics. For this purpose, create a function `prediction_perf(gt, MLE_model_predictions)` which evaluates the prediction performance of the reference models. Up to this point, you should have obtained

 - $N = 100$ samples of predictions from each model, stored in `MLE_model_predictions`.
 
 - The ground-truth FTR values from data frames `df`, stored in `gt`.

Assume predicted values for a given model are stored in a variable $P$ and its average is $\bar{P}$. The first performance measure will be the MSE, and will be calculated for each model from the expression below:

$$ MSE = \dfrac{1}{N}\sum_{i=0}^{N-1} (P_i - FTR_i)^2$$

In order to obtain the prediction accuracy for each model, you first should convert continuous prediction results into the binary form (either 1.0 or 0.0). The binarisation process will follow the piecewise function below:

  $$P_{binary, i} = \begin{cases} 1.0, & P_i \geq \bar{P}\\ 0.0, & \text{otherwise}  \end{cases} \quad \text{where} \quad i = 0, 1, \dots, 99$$

Then the percentage accuracy, $Acc\%$ is calculated as

$$ Acc\% = 100 - \sum_{i=0}^{99} |P_{binary, i} - FTR_i|$$

Following these, by using `sklearn` module methods `roc_curve()` and `auc()` find ROC curve parameters and AUC metric for each prediction model. 

In order to obtain performance analysis results in a neatly way, you then need to create a new `pandas` dataframe `df_results` which will be in the form of

```
+----+-------------+--------+--------+-------+
|    | Model       |   Acc% |    MSE |   AUC |
+====+=============+========+========+=======+
|  0 | OLS         |  77.00 | 0.1260 | 0.911 |
+----+-------------+--------+--------+-------+
|  1 | MLE-Probit  |  81.00 | 0.1086 | 0.911 |
+----+-------------+--------+--------+-------+
|  2 | MLE-Poisson |  76.00 | 0.1490 | 0.884 |
+----+-------------+--------+--------+-------+
```

Consequently, using `sklearn` method `RocCurveDisplay()`, the `prediction_perf()` function should 

 - `print` and `return` the data frame `df_results`.

**Marking for this question**
 - (2 marks) Calculating MSE and $Acc\%$ metrics correctly.
 - (3 marks) Creating and returning dataframe `df_results`.
 
with a condition that all these three operations are performed in a **fully working `prediction_perf()` function**.

In [ ]:
def prediction_perf(gt, MLE_model_predictions):
    # your code here
    return df_results